# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [88]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from api_keys import geoapify_key

In [89]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [90]:
#%%capture --no-display

# Configure the map plot
map_plot_A = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = False,
    tiles = "OpenTopoMap",
    frame_width = 500,
    frame_height = 350,
    size = "Humidity",
    scale = 0.6,
    color = "City"
)

# Display the map plot
map_plot_A

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [91]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] <24) & (ideal_weather_df["Max Temp"] >17)]
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] <5.5]
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] <2.0]
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Humidity"] <80]
# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
314,314,horsham,51.0640,-0.3272,18.40,72,0,4.12,GB,1666108422
318,318,harwich,51.9419,1.2844,17.38,69,1,3.13,GB,1666108424
363,363,narwar,25.6500,77.9000,22.35,55,0,1.29,IN,1666108449
375,375,port hedland,-20.3167,118.5667,21.03,73,0,3.09,AU,1666108455
381,381,roebourne,-20.7833,117.1333,23.48,65,0,2.95,AU,1666108458


### Step 3: Create a new DataFrame called `hotel_df`.

In [92]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.drop(axis=1, columns=["Wind Speed", "Date", "City_ID", "Max Temp", "Cloudiness"])
#hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel", '')
#hotel_df

# Display sample data
hotel_df = hotel_df.reindex(columns=["City", "Country", "Lat", "Lng", "Humidity", "Hotel"])
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel
9,saint-pierre,RE,-21.3393,55.4781,68,
152,kahului,US,20.8947,-156.4700,60,
211,laguna,US,38.4210,-121.4238,79,
240,tikaitnagar,IN,26.9500,81.5833,59,
265,san quintin,MX,30.4833,-115.9500,74,
314,horsham,GB,51.0640,-0.3272,72,
318,harwich,GB,51.9419,1.2844,69,
363,narwar,IN,25.6500,77.9000,55,
375,port hedland,AU,-20.3167,118.5667,73,
381,roebourne,AU,-20.7833,117.1333,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [136]:
# Set parameters to search for a hotel
radius = 7
params = {
    "radius":radius,
    "apiKey":geoapify_key,
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    #response=requests.get('https://api.geoapify.com/v2/places')
    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    #print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
#hotel_df
print(requests.get(base_url))

Starting hotel search
<Response [401]>


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [140]:
%%capture --no-display

# Configure the map plot
map_plot_B = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = False,
    tiles = "OpenTopoMap",
    frame_width = 500,
    frame_height = 350,
    size = "Humidity",
    scale = 0.6,
    color = "City",
    filter="Country",
    hotel_name="Hotel Name"
)

# Display the map plot
map_plot_B
# Display the map
# YOUR CODE HERE

AttributeError: module 'geoviews' has no attribute 'element'